In [1]:
import os
from datetime import datetime


PROJECT_ROOT_DIR = "/Users/gopora/MyStuff/Dev/Workspaces/Sandbox/TroubledLife"
DATASETS_DIR = os.path.join(PROJECT_ROOT_DIR, "data")
TF_LOG_DIR = os.path.join(PROJECT_ROOT_DIR, "tf_logs")
MODEL_CHECKPOINTS_DIR = os.path.join(PROJECT_ROOT_DIR, "model_checkpoints")
TRAINING_SET_DATA_FILE = "troubled_life_policy_train_data.csv"
TEST_SET_DATA_FILE = "troubled_life_policy_test_data.csv"

now = datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = "{}/run-{}/".format(TF_LOG_DIR, now)

In [2]:
import data_preparation as dp

#dp.generate_troubled_life_policy_data(no_of_policies=10000, runtime=5, file_path=os.path.join(DATASETS_DIR, TRAINING_SET_DATA_FILE))

#dp.generate_troubled_life_policy_data(no_of_policies=2000, runtime=5, file_path=os.path.join(DATASETS_DIR, TEST_SET_DATA_FILE))

policy_histories_train = \
    dp.load_troubled_life_policy_data(file_path=os.path.join(DATASETS_DIR, TRAINING_SET_DATA_FILE))

policy_histories_test = \
    dp.load_troubled_life_policy_data(file_path=os.path.join(DATASETS_DIR, TEST_SET_DATA_FILE))

policy_histories_length_train, max_policy_history_length_train = \
    dp.get_policy_history_lengths(policy_histories=policy_histories_train)

policy_histories_length_test, max_policy_history_length_test = \
    dp.get_policy_history_lengths(policy_histories=policy_histories_test)

max_policy_history_length = max(max_policy_history_length_train, max_policy_history_length_test)

In [3]:
# Pad the histories up to maximum length of both, train and test set

# policy_histories_train = \
#     dp.pad_troubled_life_policy_histories(policy_histories=policy_histories_train,
#                                           policy_histories_lengths=policy_histories_length_train,
#                                           max_policy_history_length=max_policy_history_length)
# 
# policy_histories_test = \
#     dp.pad_troubled_life_policy_histories(policy_histories=policy_histories_test,
#                                           policy_histories_lengths=policy_histories_length_test,
#                                           max_policy_history_length=max_policy_history_length)
# 
# # # Save padded data, since always generating and padding takes too long
# policy_histories_train.to_csv(path_or_buf=os.path.join(DATASETS_DIR, TRAINING_SET_DATA_FILE))
# policy_histories_test.to_csv(path_or_buf=os.path.join(DATASETS_DIR, TEST_SET_DATA_FILE))

# Extract features and labels from dataset as numpy.ndarray(s)
binary_classification = True

train_labels, train_features, train_seq_lengths =\
    dp.prepare_labels_features_lengths(policy_histories=policy_histories_train,
                                       policy_histories_lengths=policy_histories_length_train, 
                                       max_policy_history_length=max_policy_history_length,
                                       binary_classification=binary_classification)
test_labels, test_features, test_seq_lengths =\
    dp.prepare_labels_features_lengths(policy_histories=policy_histories_test,
                                       policy_histories_lengths=policy_histories_length_test, 
                                       max_policy_history_length=max_policy_history_length,
                                       binary_classification=binary_classification)

train_data = dp.TrainDataSet(train_labels=train_labels, train_features=train_features, train_seq_lengths=train_seq_lengths)

In [4]:
import tensorflow as tf
import gan as gan


g_learning_rate = 0.001
d_real_learning_rate = 0.0001
d_fake_learning_rate = 0.0001
noise_dimensions = 100
batch_size = 200
g_n_outputs = 2
d_n_inputs = 2
d_n_outputs = 2 if binary_classification else max_policy_history_length

tf.reset_default_graph()
tf.set_random_seed(42)

noise = tf.placeholder(tf.float32, [None, max_policy_history_length, noise_dimensions], name='noise')
noise_seq_length = tf.placeholder(tf.int32, [None], name='noise_seq_length')

X = tf.placeholder(tf.float32, [None, max_policy_history_length, d_n_inputs], name="X")
y = tf.placeholder(tf.int32, [None], name="y")
X_seq_length = tf.placeholder(tf.int32, [None], name="X_seq_length")

g_data = gan.generator(noise=noise, n_outputs=g_n_outputs, seq_length=noise_seq_length)

logits_d_for_X, y_pred_X = gan.discriminator(X=X, seq_length=X_seq_length, n_outputs=d_n_outputs)
logits_d_for_g, y_pred_g = gan.discriminator(X=g_data, seq_length=noise_seq_length, n_outputs=d_n_outputs, reuse=True)

g_loss, accuracy_g = gan.generator_loss(batch_size=batch_size, logits=logits_d_for_g)
d_loss_real, accuracy_d_real = gan.discriminator_loss_real(logits=logits_d_for_X, y=y)
d_loss_fake, accuracy_d_fake = gan.discriminator_loss_fake(batch_size=batch_size, logits=logits_d_for_g)

g_trainer = gan.generator_trainer(g_learning_rate, g_loss)
d_trainer_real = gan.discriminator_trainer_real(d_real_learning_rate, d_loss_real)
d_trainer_fake = gan.discriminator_trainer_fake(d_fake_learning_rate, d_loss_fake)

tf.summary.scalar('Generator_loss', g_loss)
tf.summary.scalar('Discriminator_loss_real', d_loss_real)
tf.summary.scalar('Discriminator_loss_fake', d_loss_fake)

merged = tf.summary.merge_all()
writer = tf.summary.FileWriter(log_dir, tf.get_default_graph())

In [5]:
import numpy as np


sess = tf.Session()

sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())

saver = tf.train.Saver()

# n_epochs = 100
# 
# # Pre-train discriminator
# for epoch in range(n_epochs):
#     for batch in range(train_data.num_examples // batch_size):
#         noise_batch = np.random.normal(0, 1, size=[batch_size, max_policy_history_length, noise_dimensions])
#         noise_seq_length_batch = np.full(batch_size, max_policy_history_length)
# 
#         y_batch, X_batch, X_seq_length_batch = train_data.next_batch(batch_size)
# 
#         _, __, dLossReal, dLossFake, acc_real_train, acc_fake_train = \
#             sess.run([d_trainer_real, d_trainer_fake, d_loss_real, d_loss_fake, accuracy_d_real, accuracy_d_fake],
#                      {X: X_batch, y: y_batch, X_seq_length: X_seq_length_batch,
#                       noise: noise_batch, noise_seq_length: noise_seq_length_batch})
# 
#     print("Epoch:", epoch, "dLossReal:", dLossReal, "acc_real_train:", acc_real_train,
#           "dLossFake:", dLossFake, "acc_fake_train:", acc_fake_train)

saver.save(sess, os.path.join(MODEL_CHECKPOINTS_DIR, "discriminator.ckpt"))

Epoch: 0 dLossReal: 12.1091 acc_real_train: 0.835 dLossFake: 0.0278049 acc_fake_train: 1.0


Epoch: 1 dLossReal: 11.6146 acc_real_train: 0.935 dLossFake: 3.11428e-05 acc_fake_train: 1.0


Epoch: 2 dLossReal: 2.79858 acc_real_train: 0.92 dLossFake: 2.27673e-05 acc_fake_train: 1.0


Epoch: 3 dLossReal: 0.501413 acc_real_train: 0.925 dLossFake: 1.23367e-05 acc_fake_train: 1.0


Epoch: 4 dLossReal: 2.18449 acc_real_train: 0.935 dLossFake: 5.17305e-06 acc_fake_train: 1.0


...

Epoch: 96 dLossReal: 0.0347649 acc_real_train: 0.995 dLossFake: 0.0 acc_fake_train: 1.0


Epoch: 97 dLossReal: 0.0614429 acc_real_train: 0.985 dLossFake: 0.0 acc_fake_train: 1.0


Epoch: 98 dLossReal: 0.051425 acc_real_train: 0.99 dLossFake: 5.96046e-10 acc_fake_train: 1.0


Epoch: 99 dLossReal: 0.0663511 acc_real_train: 0.985 dLossFake: 0.0 acc_fake_train: 1.0


'/Users/gopora/MyStuff/Dev/Workspaces/Sandbox/TroubledLife/model_checkpoints/discriminator.ckpt'

In [8]:
saver.restore(sess, os.path.join(MODEL_CHECKPOINTS_DIR, "discriminator.ckpt"))

#Train generator and discriminator together
n_epochs = 200

for epoch in range(n_epochs):
    for batch in range(train_data.num_examples // batch_size):
        noise_batch = np.random.normal(0, 1, size=[batch_size, max_policy_history_length, noise_dimensions])
        noise_seq_length_batch = np.full(batch_size, max_policy_history_length)
        
        # y_batch, X_batch, X_seq_length_batch = train_data.next_batch(batch_size)
        # 
        # # Train discriminator on both real and fake data
        # _, __, dLossReal, dLossFake, acc_real_train, acc_fake_train = \
        #     sess.run([d_trainer_real, d_trainer_fake, d_loss_real, d_loss_fake, accuracy_d_real, accuracy_d_fake],
        #              {X: X_batch, y: y_batch, X_seq_length: X_seq_length_batch,
        #               noise: noise_batch, noise_seq_length: noise_seq_length_batch})
        # 
        # # Train generator
        # noise_batch = np.random.normal(0, 1, size=[batch_size, max_policy_history_length, noise_dimensions])
    
        #_, gLoss, acc_g, gData = sess.run([g_trainer, g_loss, accuracy_g, g_data], 
         #                            feed_dict={noise: noise_batch, noise_seq_length: noise_seq_length_batch})

        gLoss, acc_g, gData = sess.run([g_loss, accuracy_g, g_data], 
                                     feed_dict={noise: noise_batch, noise_seq_length: noise_seq_length_batch})

    # print("Epoch:", epoch, "dLossReal:", dLossReal, "acc_real_train:", acc_real_train,
    #       "dLossFake:", dLossFake, "acc_fake_train:", acc_fake_train,
    #       "gLoss:", gLoss, "acc_g:", acc_g)

    print("Epoch:", epoch, "gLoss:", gLoss, "acc_g:", acc_g)
    print(gData[0])


INFO:tensorflow:Restoring parameters from /Users/gopora/MyStuff/Dev/Workspaces/Sandbox/TroubledLife/model_checkpoints/discriminator.ckpt


Epoch: 0 gLoss: 27.2138 acc_g: 0.0
[[ 0.08572766 -0.3339045 ]
 [ 0.63805389 -0.53719294]
 [ 0.29396552 -0.88731331]
 [ 0.37408763 -0.81510913]
 [ 0.52255464 -0.80806035]
 [ 0.22934756 -0.72426963]
 [ 0.55368775 -0.84412819]
 [ 0.46802622 -0.76123506]
 [ 0.08683917 -0.49970955]
 [ 0.14626271 -0.53411072]
 [-0.04495898 -0.98848557]
 [ 0.5454247  -0.78546   ]
 [ 0.40206867 -0.31760377]
 [-0.01882824 -0.64912558]
 [ 0.3344661  -0.73534912]]


Epoch: 1 gLoss: 27.0327 acc_g: 0.0
[[ 0.21139652 -0.29061091]
 [ 0.20495433 -0.09223257]
 [-0.2240018  -0.4724223 ]
 [ 0.15971807 -0.79548502]
 [ 0.1683991  -0.61861038]
 [ 0.25871554 -0.67234921]
 [ 0.6776123  -0.38401467]
 [ 0.21735084 -0.83035386]
 [ 0.31422165 -0.39548212]
 [-0.06340641 -0.55020905]
 [ 0.13907635 -0.6200394 ]
 [ 0.04049653 -0.82181567]
 [ 0.26221937 -0.69684851]
 [ 0.47207761 -0.97188914]
 [-0.20633332 -1.45616829]]


Epoch: 2 gLoss: 27.2686 acc_g: 0.0
[[ 0.12425978 -0.40530032]
 [ 0.36337101 -0.31753972]
 [ 0.19453563 -0.72451615]
 [-0.10031834 -0.80538511]
 [ 0.07437791 -0.61868203]
 [ 0.22540016 -0.61645567]
 [ 0.37888542 -0.40647614]
 [-0.09915863 -0.93858409]
 [ 0.48669076 -0.60947013]
 [ 0.19616118 -0.99185592]
 [ 0.49608505 -0.71847445]
 [-0.1099067  -0.61188257]
 [ 0.04296386 -0.97055423]
 [ 0.21626809 -0.79605448]
 [ 0.40841645 -0.88203043]]


Epoch: 3 gLoss: 27.2622 acc_g: 0.0
[[ 0.10786431 -0.18153486]
 [ 0.2794888  -0.43915567]
 [ 0.26810855 -0.41593456]
 [ 0.51244843 -0.42175919]
 [-0.40007627 -0.76659095]
 [ 0.27112916 -0.42232442]
 [ 0.0794186  -0.69545227]
 [ 0.10316613 -1.03998971]
 [ 0.42056111 -0.59607339]
 [ 0.15164855 -1.03368974]
 [-0.00118825 -0.78469026]
 [ 0.31138164 -0.55444914]
 [ 0.60503161 -0.94713175]
 [ 0.49692386 -0.75040162]
 [ 0.57851624 -0.54554391]]


...

Epoch: 10 gLoss: 26.8122 acc_g: 0.0
[[ 0.05572231 -0.19995378]
 [-0.12898763 -0.31791419]
 [ 0.42750391 -0.17538781]
 [ 0.2387474  -0.40012622]
 [-0.01133863 -0.75270337]
 [ 0.14660308 -0.83726776]
 [ 0.21434623 -0.96419942]
 [ 0.67584336 -1.25266969]
 [ 0.4705182  -0.43296486]
 [ 0.33817694 -0.60163242]
 [ 0.32286111 -0.82884645]
 [ 0.6769374  -0.72085416]
 [ 0.09604347 -1.02022815]
 [ 0.11708333 -0.72979271]
 [ 0.26681435 -0.74625146]]


Epoch: 11 gLoss: 26.9753 acc_g: 0.0
[[ 0.05612445 -0.40019202]
 [ 0.12506317 -0.39186239]
 [ 0.15520023 -0.30676928]
 [ 0.17124183 -0.45835489]
 [-0.19189408 -0.59409612]
 [ 0.375824   -0.4521262 ]
 [-0.18366998 -0.78132963]
 [-0.00360887 -0.68383956]
 [ 0.10498755 -0.4498415 ]
 [ 0.19682099 -0.63520873]
 [ 0.40187699 -0.82074749]
 [ 0.48558858 -0.61852229]
 [ 0.03818099 -0.82386965]
 [ 0.49374017 -0.05204661]
 [ 0.24864718 -0.55045104]]


Epoch: 12 gLoss: 27.0637 acc_g: 0.0
[[ 0.09701312 -0.45761207]
 [ 0.08877711 -0.38476765]
 [ 0.04597384 -0.84569132]
 [ 0.20852922 -0.45662409]
 [ 0.51576084 -0.39409697]
 [ 0.05130914 -0.66716957]
 [ 0.31294909 -0.47139785]
 [ 0.32007873 -0.79022384]
 [ 0.75308883 -0.61952114]
 [-0.07400514 -0.67902344]
 [ 0.42937791 -1.03568971]
 [ 0.07162699 -0.77885711]
 [ 0.40470389 -0.60806668]
 [ 0.05202305 -0.746378  ]
 [ 0.12367684 -0.39067268]]


Epoch: 13 gLoss: 26.9933 acc_g: 0.0
[[ 0.13962208 -0.2078197 ]
 [ 0.15314154 -0.22034781]
 [-0.00280667 -0.08769958]
 [ 0.27719909 -0.36568597]
 [-0.11637765 -0.58227247]
 [ 0.07935384 -0.63251549]
 [ 0.80671972 -0.62144566]
 [-0.04120871 -1.07031083]
 [ 0.43696886 -1.19712019]
 [ 0.39208174 -1.25708938]
 [ 0.32360703 -0.53142059]
 [ 0.47028148 -0.54137635]
 [ 0.14439075 -0.53456342]
 [ 0.32504463 -0.32838565]
 [ 0.47595501 -0.54781795]]


KeyboardInterrupt: 